# Трансферное обучение в PyThorch

## Задача

Cоздать свёрточную модель трансферного обучения для классификации бетона с трещиной (Positive mark) и бетона без трещины (Negative mark)  
Как было показано в [1_Models_for_Transfer_Learning_PyTorch.ipynb](https://github.com/Aleks-Zink/Pet_Projects/blob/main/2_Concrete/1_Models_for_Transfer_Learning_PyTorch.ipynb), за основу будет взята модель VGG16

## Данные

40 000 цветных картинок бетона размером 227x227 пикселей, 20 000 из которых с стрещиной, другие 20 000 целый  
Данные взяты из курса [AI Capstone Project with Deep Learning](https://www.coursera.org/learn/ai-deep-learning-capstone?specialization=ai-engineer), явлюющийся заключительным курсом [IBM AI Engineering Professional Certificate](https://www.coursera.org/professional-certificates/ai-engineer) на сайте [coursera.org](https://www.coursera.org/)  
[Данные](https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip) 

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip

--2023-04-28 21:22:42--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245259777 (234M) [application/zip]
Saving to: ‘concrete_crack_images_for_classification.zip’

concrete_crack_imag 100%[===================>] 233.90M  26.8MB/s    in 8.3s    

2023-04-28 21:22:51 (28.1 MB/s) - ‘concrete_crack_images_for_classification.zip’ saved [245259777/245259777]



In [2]:
!mkdir ./data
!mkdir ./data/concrete
!mkdir ./models

In [ ]:
!unzip concrete_crack_images_for_classification.zip -d ./data/concrete

## Расчёты

In [4]:
import os

from PIL import Image

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models

from tqdm import tqdm

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [5]:
class Concrete(Dataset):
    def __init__(self, train=True, train_size=0.75, transform=None):

        self.transform = transform

        # директории классов
        directory = "./data/concrete/"
        positive = "Positive"
        negative = "Negative"

        # путь к классам
        positive_path = os.path.join(directory, positive)
        negative_path = os.path.join(directory, negative)

        # путь к каждому положительному объекту
        positive_files = [os.path.join(positive_path, file) for file in os.listdir(positive_path) if file.endswith(".jpg")]
        positive_files.sort()

        # путь к каждому негативному объекту
        negative_files = [os.path.join(negative_path, file) for file in os.listdir(negative_path) if file.endswith(".jpg")]
        negative_files.sort()

        # пути к каждому объекту (чётные индексы - положительные классы, нечётные индексы - отрицательные классы)
        self.all_files = []
        for pair in zip(positive_files, negative_files):
            self.all_files.extend(pair)
        
        # всего объектов в данных
        length = len(self.all_files)

        # целевое обозначение каждого объекта 
        self.all_targets = torch.zeros(length, dtype=torch.long)
        self.all_targets[::2] = 1

        # индекс границы между данными обечения и данными валидации
        border = int(length * train_size)

        # разделение данных на тестовую валидационную выборки
        if train:
            self.all_files = self.all_files[:border]
            self.all_targets = self.all_targets[:border]
        else:
            self.all_files = self.all_files[border:]
            self.all_targets = self.all_targets[border:]
        
        self.len = len(self.all_targets)

    def __len__(self):
        return self.len

    def __getitem__(self, ind):
        
        # загрузка оъекта и его обозначения
        image = Image.open(self.all_files[ind])
        y = self.all_targets[ind]

        # трансформация объекта, если это необходимо
        if self.transform:
            image = self.transform(image)

        return image, y

In [6]:
def train(model, data_train, data_val, val_len, device='cpu', epochs=3, lr=0.001, save_model=False, acc_max=0):
    
    # инициализация оптимизатора и функции ошибки
    optimizer = optim.Adam(params=[parameter for parameter in model.parameters() if parameter.requires_grad], lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # перевод модель на устройство для расчётов
    model.to(device)

    for epoch in range(epochs):
        
        # перевод модели в режим обучения
        model.train()

        # цикл обучения
        for x, y in tqdm(data_train, desc=f"Train epoch {epoch + 1: >3}"):

            # перевод данных на устройсто для расчётов         
            x, y = x.to(device), y.to(device)
            
            # обнуление градиента
            optimizer.zero_grad()
            
            # расчёт ошибки
            y_hat = model(x)
            loss = criterion(y_hat, y)

            # шаг обучения
            loss.backward()
            optimizer.step()

        # перевод модели в режим оценки
        model.eval()

        # число корректных ответов
        correct = 0

        # цикл оценки
        for x, y in tqdm(data_val, desc=f"Val epoch {epoch + 1: >5}"):

            # перевод данных на устройсто для расчётов 
            x, y = x.to(device), y.to(device)

            # сбор корректных ответов в батче
            with torch.no_grad():
                y_hat = model(x)
            correct += (y_hat.argmax(dim=1) == y).sum().item()

        # расчёт точности
        accuracy = correct / val_len
        
        print(f"\nAccuracy: {accuracy * 100:.2f}%\n")

        # сохранение наилучшей модели
        if save_model and (acc_max < accuracy):
            acc_max = accuracy
            torch.save(model, './models/VGG16_for_Concrete_PyTorch.pt')
            print("=== Модель сохранена ===\n")

    # перевод модели на процессор
    model.to("cpu")

    return accuracy, acc_max

In [7]:
# веса модели VGG16
weights = models.VGG16_Weights.IMAGENET1K_V1

# инициализация модели
model = models.vgg16(weights=weights)

# заморозка весов свёрточной модели
for param in model.parameters():
    param.requires_grad = False 

# замена полносвязных слоёв в конце
model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
model.classifier = nn.Linear(512, 2)

model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 76.6MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
# размер выборки для обучения
train_size = 0.75
# преобразователь данных
transform = weights.transforms()
# размер пакета данных
batch_size = 100
# перемешивать данные
shuffle = True


# инициализация обучающих данных
data_train = Concrete(train=True, train_size=train_size, transform=transform)
# инициализация валидационных данных
data_val = Concrete(train=False, train_size=train_size, transform=transform)


# размер валидационной выборки
val_len = len(data_val)


# пакетированные данные для тренировки
data_train = DataLoader(dataset=data_train, batch_size=batch_size, shuffle=shuffle)
# пакетированные данные для валидации
data_val = DataLoader(dataset=data_val, batch_size=batch_size, shuffle=shuffle)

In [9]:
# обучение модели
accuracy, acc_max = train(model,
                          data_train=data_train,
                          data_val=data_val,
                          val_len=val_len,
                          device=device,
                          epochs=3,
                          lr=1e-3,
                          save_model=False,
                          acc_max=0)

Val epoch     1: 100%|██████████| 100/100 [01:31<00:00,  1.10it/s]



Accuracy: 99.13%



Val epoch     2: 100%|██████████| 100/100 [01:31<00:00,  1.10it/s]



Accuracy: 99.25%



Val epoch     3: 100%|██████████| 100/100 [01:31<00:00,  1.09it/s]


Accuracy: 99.36%



Для повышения точности разморозим все веса модели и дообучим всю модель с уменьшенной скоростью, для подстраивания свёрточных слоёв под наши данные

In [10]:
# разморозка весов
for param in model.parameters():
    param.requires_grad = True

# обучение модели c уменьшенной скоростью обучения
accuracy, acc_max = train(model,
                          data_train=data_train,
                          data_val=data_val,
                          val_len=val_len,
                          device=device,
                          epochs=3,
                          lr=1e-4,
                          save_model=True,
                          acc_max=accuracy)

Val epoch     1: 100%|██████████| 100/100 [01:30<00:00,  1.10it/s]



Accuracy: 99.88%

=== Модель сохранена ===



Val epoch     2: 100%|██████████| 100/100 [01:34<00:00,  1.05it/s]



Accuracy: 99.54%



Val epoch     3: 100%|██████████| 100/100 [01:31<00:00,  1.09it/s]


Accuracy: 99.85%



In [11]:
# загружаем модель с наилучшей точностью
model = torch.load('./models/VGG16_for_Concrete_PyTorch.pt')

# вычисление числа параметров модели
number_of_param = 0
for param in model.parameters():
    number_of_param += param.numel()

print(f"Получаем модель c:\n\tОбъёмом {number_of_param * 4 / (1024 ** 2):.2f} Мб\n\tТочностью {acc_max * 100:.2f}%")

Получаем модель c:
	Объёмом 56.14 Мб
	Точностью 99.88%


Для дальнейшего использования модели, входные данные должны иметь форму (B, C, H, W)  
Где:
 - B: количество фото для анализа
 - С = 3: цветовых каналов
 - H = 224: высота фото
 - W = 224: ширина фото

## Результаты

Была получена свёрточная модель трансферного обучения на базе модели VGG16 с размером модели в ~56 Мб.  
Получившаяся модель с точностью ~99.88% может классифицировать фотографии бетона с трещиной и бетона без трещины

## Опыт работы в PyTorch

Плюсы:
 + Простая установка  
 + Простота освоения
 + Минимализм
 
Минусы:
 + Большинство вещей делается вручную (к примеру: цикл обучения, метрики для оценки моделей)

Лично для себя, на данный момент, выбираю PyTorch в качестве основной бибилиотеки для NN, так как больше опыта в её использовании